## Interactuando con Github

In [2]:
from github import Github
import os
from dotenv import load_dotenv, find_dotenv
from time import sleep
import re
import unicodedata
import json
import pickle

load_dotenv(find_dotenv())


git_key = os.getenv("GITHUB_ADMIN_TOKEN")
g = Github(git_key)

def download_github_data():
    with open("files/github_books.pkl", "wb") as file:
        books = _get_books()
        pickle.dump(books, file)


def load_github_data():
    books = []
    with open("files/github_books.pkl", "rb") as file:
        try:
            while True:
                books.append(pickle.load(file))
        except EOFError:
            return books[0]
        
def _get_books():
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Devuelve:
        listado de gitbook de la organización CATEDU.
    """
    global g
    repos = list(g.get_organization("catedu").get_repos())
    gitbooks = [repo for repo in repos if _has_summary(repo)]
    return gitbooks

def _has_summary(repo):
    """Comprueba si un repositorio tiene archivo SUMMARY.md, indicativo de que
    se trata de un Gitbook.

    Parámetros:
        repo (Objeto PyGithub que representa un repositorio)

    Devuelve:
        True or False (Boolean) indicando si tiene o no SUMMARY.md
    """
    try:
        contents = [content.name for content in repo.get_contents("")]
        it_has_summary = "SUMMARY.md" in contents
        return it_has_summary
    except:
        print(f"######### El repo {repo.name} falla #############")
        
def repos_data():
    books = load_github_data()
    data = [
        {
            "repo_name": book.name,
            "master_url": book.html_url,
            "gh-pages": f"https://catedu.github.io/{book.name}/",
        }
        for book in books
    ]
    return data

In [2]:
download_github_data()

In [3]:
books = load_github_data()

In [4]:
books[30]

Repository(full_name="catedu/robotica-educativa-con-mbot-II")

## Importar libros de Github a Wagtail

Para esta tarea tenemos que:

1. Descargar el SUMMARY.md de cada libro
2. Por cada apartado guardar:
  * Orden
  * Nivel de subapartado
  * Título
  * Ruta del archivo.md
3. Cambiar de rama a gh-pages:
  * Obtener el html de cada archivo
  * Descargar imágenes.
    * Si es ruta absoluta, comprobar si viene del mismo repo y bajar
  * Guardar html, sustituyendo la ruta de los archivos locales.
4. Crear páginas en Wagtail dividiendo html y bloques de código

### Obteniendo el índice

#### Mediante API

In [37]:
import re
import pprint

def get_summary(book):
    summary = book.get_contents("SUMMARY.md")
    summary = summary.decoded_content.decode().split("\n")
    summary = [item for item in summary 
               if item != "" 
               and item != "# Summary"
               and not item.startswith("* [Créditos]")
              ]
    return summary

def save_index(summary_string):
    ruta_y_titulo = re.compile(r"\* \[(.*)\]\((.*)\)")
    index = []
    # Calculando el nivel de indentación según el número de espacios
    niveles = {"0": 2, "2": 3, "4": 4, "6": 5}

    for line in summary:
        if line.startswith("##"):
            item = {"nivel": 1}
            item["ruta_archivo"] = None
            item["titulo"] = line.strip("## ")
            index.append(item)
        else:
            leading_spaces = str(len(line) - len(line.lstrip(" ")))
            item = {"nivel": niveles[leading_spaces]}
            item["ruta_archivo"] = ruta_y_titulo.findall(line)[0][1]
            item["titulo"] = ruta_y_titulo.findall(line)[0][0]
            index.append(item)
    
    return index

summary = get_summary(books[30])
save_index(summary)

[{'nivel': 2, 'ruta_archivo': 'README.md', 'titulo': 'Introducción'},
 {'nivel': 1, 'ruta_archivo': None, 'titulo': '1. Veamos ejemplos'},
 {'nivel': 2, 'ruta_archivo': 'index0.md', 'titulo': 'M1 mBot plus'},
 {'nivel': 2, 'ruta_archivo': 'repaso_de_mbot.md', 'titulo': 'Repaso de mBot'},
 {'nivel': 2, 'ruta_archivo': 'robot_polilla.md', 'titulo': 'Robot polilla'},
 {'nivel': 2,
  'ruta_archivo': 'comedinosaurio.md',
  'titulo': 'Come-dinosaurio'},
 {'nivel': 2,
  'ruta_archivo': 'seguir_circuito_y_ultrasonidos.md',
  'titulo': 'Seguir circuito y ultrasonidos'},
 {'nivel': 2, 'ruta_archivo': 'evitar_caerse.md', 'titulo': 'Evitar caerse'},
 {'nivel': 2, 'ruta_archivo': 'radar.md', 'titulo': 'Radar'},
 {'nivel': 2,
  'ruta_archivo': 'matemticas_y_robtica.md',
  'titulo': 'Matemáticas y robótica'},
 {'nivel': 2,
  'ruta_archivo': 'mbot_en_infantil.md',
  'titulo': 'mBot en Infantil'},
 {'nivel': 2,
  'ruta_archivo': 'al_infinito_y_ms_all.md',
  'titulo': 'Al infinito y más allá...'},
 {'ni

#### Scrapeando

In [6]:
import mechanicalsoup
browser = mechanicalsoup.StatefulBrowser()

browser.open("https://catedu.github.io/introduccion-a-python/")
index = browser.page.find("ul", class_="summary").find_all("li")
content = browser.page.find("section", class_="markdown-section")

In [7]:
tag = browser.page.find("ul", class_="summary")
index = [li.attrs for li in tag.find_all("li") if len(li.attrs.keys()) > 0]

In [8]:
for page in index:
    if "chapter" in page["class"]:
        base_url = f"https://catedu.github.io/introduccion-a-python/"
        url = page["data-path"] if page["data-path"] != "./" else ""
        browser.open(base_url + url)
        page["content"] = browser.page.find("section", class_="markdown-section").decode_contents()

### Descargando imágenes

#### Mediante API

In [20]:
contents = books[0].get_contents("")

while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(books[0].get_contents(file_content.path))
    else:
        print(file_content.download_url)

https://raw.githubusercontent.com/catedu/abies-lectores/master/.gitignore
https://raw.githubusercontent.com/catedu/abies-lectores/master/.nojekyll
https://raw.githubusercontent.com/catedu/abies-lectores/master/FOOTER.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/LICENSE
https://raw.githubusercontent.com/catedu/abies-lectores/master/README.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/SUMMARY.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/book.json
https://raw.githubusercontent.com/catedu/abies-lectores/master/consultar_catlogos_de_otras_bibliotecas.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/consultar_el_catlogo_de_fondos_de_la_biblioteca_de_mi_centro.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/crditos.md
https://raw.githubusercontent.com/catedu/abies-lectores/master/darme_de_alta_en_abiesweb_para_usar_la_biblioteca_de_mi_centro_escolar.md
https://raw.githubusercontent.com/catedu/abie

In [23]:
os.system("you-get -o files -O test https://raw.githubusercontent.com/catedu/abies-lectores/master/img/consultarOpiniones.JPG")

0

#### Scrapeando

In [9]:
browser.open("https://catedu.github.io/introduccion-a-python/reflexin_inicial.html")

<Response [200]>

In [10]:
img = browser.page.find_all("img")[0]["src"]

In [13]:
img

'img/usuario_vs_informaticos_thumb[3].png'

In [15]:
base_url = "https://catedu.github.io/introduccion-a-python/"
os.system(f'you-get -o files -O {img.split("/")[1]} {base_url}{img}')

0

In [130]:
f"you-get {base_url}{img}"

'you-get https://catedu.github.io/introduccion-a-python/img/usuario_vs_informaticos_thumb[3].png'